<a href="https://colab.research.google.com/github/sandy273040/Text_Mining/blob/main/lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm requests multitasking retry bs4 pandas numpy xlrd openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
import numpy as np
import pickle
import itertools
from collections import Counter
from nltk.util import ngrams
from gensim import corpora
from gensim import models
from gensim import similarities

###Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Read File

In [ ]:
import os
path = os.path.abspath(os.getcwd())
restaurant_df = pd.read_csv(path + '/drive/Shareddrives/文字探勘/final_project/restaurant_final_df.csv', index_col=[0])
restaurant_df

,restaurant_id,restaurant_name,length,ave_rating,comment_amount,topic_id,topic_words
0,0,茱莉金牛排餐酒館,634,4.5,967,26,"['""牛排"" ', '""服務"" ', '""龍蝦"" ', '""用餐"" ', '""餐點"" ', ..."
1,1,Okaeri お帰り 你回來啦拉麵,840,4.3,2769,28,"['""拉麵"" ', '""湯頭"" ', '""豚骨"" ', '""叉燒"" ', '""味道"" ', ..."
2,2,和牛涮 日式鍋物放題 台北忠孝東店,840,4.8,21588,17,"['""服務"" ', '""新鮮"" ', '""用餐"" ', '""店員"" ', '""態度"" ', ..."
3,3,狗一下居食酒屋 忠孝店,840,4.8,18200,17,"['""服務"" ', '""新鮮"" ', '""用餐"" ', '""店員"" ', '""態度"" ', ..."
4,4,蘇阿姨比薩屋,840,4.1,8271,32,"['""炸雞"" ', '""小籠包"" ', '""披薩"" ', '""服務"" ', '""湯包"" ',..."
...,...,...,...,...,...,...,...
101,101,北門鳳李冰,840,4.6,2279,34,"['""芋頭"" ', '""花生"" ', '""鳳梨"" ', '""古早味"" ', '""冰品"" ',..."
102,102,蛤?! Huh Pot,840,4.8,4614,23,"['""粉圓"" ', '""蛤蜊"" ', '""糖水"" ', '""新鮮"" ', '""服務"" ', ..."
103,103,小李子清粥小菜,840,3.8,5029,37,"['""服務"" ', '""餐點"" ', '""用餐"" ', '""菜色"" ', '""選擇"" ', ..."
104,104,温州街蘿蔔絲餅達人,840,4.2,4100,30,"['""蘿蔔絲"" ', '""蘿蔔絲餅"" ', '""排隊"" ', '""提供"" ', '""蔥油餅""..."


###Vectorization

In [ ]:
comments = pickle.load(open(path+'/drive/Shareddrives/文字探勘/final_project/Django/comments_final', 'rb'))

In [ ]:
dictionary = corpora.Dictionary(comments)
corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in comments]
# for doc in corpus:
#    print([[dictionary[id], freq] for id, freq in doc])

In [ ]:
model = models.TfidfModel(corpus, id2word = dictionary)#smartirs='ntc', 

In [ ]:
#filter low tfidf word
low_value = 0.025

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    low_value_words = [id for id, value in model[bow] if value < low_value]
    new_bow = [b for b in bow if b[0] not in low_value_words]

    #reassign        
    corpus[i] = new_bow

In [ ]:
word = list()
rest_word = list()
for doc in model[corpus]:#document tfidf vector
  # print([(dictionary[id], freq) for id, freq in doc])
  word.append([dictionary[id] for id, freq in doc])
  rest_word.append([freq for id, freq in doc])

In [ ]:
tfidf = model[corpus]
tfidf

###LDA

####Compute coherence score to determine number of topics

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=25, passes=20)#好吃、可以、非常、不錯、真的
topic_list = lda.print_topics(25)

In [ ]:
def get_topic_word(topic_list):
  '''
  input: top 10 word distribution of each topic
  output: formatted topic-word distribution using dictionary
  '''
  topic_words = dict()
  for topic_id, topic_word in enumerate(topic_list):
    topic_words[topic_id] = [item.split('*')[1] for item in topic_word[1].split('+')]
    # print(' '.join([item.split('*')[1] for item in i[1].split('+')]))
  return topic_words

In [ ]:
topic_words = get_topic_word(topic_list)
topic_words

{0: ['"蛋糕" ',
  '"服務" ',
  '"草莓" ',
  '"檸檬" ',
  '"店員" ',
  '"甜點" ',
  '"喜歡" ',
  '"大理石" ',
  '"態度" ',
  '"奶油"'],
 1: ['"酸菜" ',
  '"服務" ',
  '"蔥油餅" ',
  '"餐廳" ',
  '"味道" ',
  '"白肉鍋" ',
  '"用餐" ',
  '"推薦" ',
  '"起司" ',
  '"喜歡"'],
 2: ['"蘿蔔絲" ',
  '"蘿蔔絲餅" ',
  '"排隊" ',
  '"蔥油餅" ',
  '"蛋餅" ',
  '"內餡" ',
  '"味道" ',
  '"餅皮" ',
  '"蘿蔔" ',
  '"豆沙餅"'],
 3: ['"牛排" ',
  '"服務" ',
  '"餐點" ',
  '"用餐" ',
  '"龍蝦" ',
  '"餐廳" ',
  '"麵包" ',
  '"環境" ',
  '"推薦" ',
  '"沙拉"'],
 4: ['"牛肉麵" ',
  '"味道" ',
  '"價格" ',
  '"湯頭" ',
  '"牛肉" ',
  '"小菜" ',
  '"紅燒" ',
  '"服務" ',
  '"粉蒸" ',
  '"蛤蜊"'],
 5: ['"餐點" ',
  '"鬆餅" ',
  '"環境" ',
  '"咖啡" ',
  '"舒芙蕾" ',
  '"服務" ',
  '"適合" ',
  '"味道" ',
  '"店員" ',
  '"喜歡"'],
 6: ['"炸雞" ',
  '"韓式" ',
  '"店員" ',
  '"年糕" ',
  '"服務" ',
  '"態度" ',
  '"韓國" ',
  '"洋釀" ',
  '"原味" ',
  '"時間"'],
 7: ['"服務" ',
  '"壽司" ',
  '"用餐" ',
  '"夜市" ',
  '"和牛" ',
  '"新鮮" ',
  '"選擇" ',
  '"品質" ',
  '"肉質" ',
  '"價位"'],
 8: ['"套餐" ',
  '"服務" ',
  '"餐點" ',
  '"雙人" ',
  '"用餐" ',
  '"份量" ',
  '"味道" ',
  '"菜色

###Compute cosine similarity among each of the two restaurants

In [ ]:
index = gensim.similarities.MatrixSimilarity(model[corpus])#lda[corpus] returns topic distribution for each doc; model[corpus] reutrns tfidf vector for each doc

NameError: ignored

####Load similarity index

In [ ]:
index = pickle.load(open('/content/drive/Shareddrives/文字探勘/final_project/Django/similarity.pkl','rb'))

###Restaurant ranking based on a given topic keyword

In [ ]:
input_keyword = "正餐 日式 生魚片 評價"

In [ ]:
def preprocess_input(input_keyword):
  to_token = input_keyword.split()
  bow_vector = dictionary.doc2bow(to_token)
  bow_vector_ordered = sorted(bow_vector, key=lambda x: x[1], reverse=True)# returns (word id, freq of word)
  # print(bow_vector_ordered[:10])
  topic_distrib = lda[bow_vector_ordered]#topic distribution of the input_keyword
  # print(topic_distrib)

  return bow_vector_ordered

In [ ]:
def retrieve(preference):
  '''
  input: latent preference vector
  output: top five similar restaurants
  '''
  sims = index[preference]
  recommended_rest_id = sims.argsort()[-2:-7:-1]

  recommended = [restaurant_df[restaurant_df['restaurant_id'] == id]['restaurant_name'].tolist()[0] for id in recommended_rest_id]#retrieve five restaurants

  #Sorting by ave_rating
  recommended_ordered_index = np.argsort([restaurant_df[restaurant_df['restaurant_name'] == restaurant]['ave_rating'].tolist()[0] for restaurant in recommended])
  if len(recommended_ordered_index) > 5: recommended_ordered_index = recommended_ordered_index[:5]
  recommended = [recommended[i] for i in recommended_ordered_index]
  return recommended

In [ ]:
input_vector = preprocess_input(input_keyword)
print('Top relevant restaurants', '-----', retrieve(input_vector), sep='\n')

Top relevant restaurants
-----
['惡犬食堂', '彩虹座Buffet (福華大飯店)', '漢來海港 敦化店', '遠東Café (香格里拉遠東飯店)', '青田七六']


###Recommend restaurants based on selected restaurant's corresponding topic

In [ ]:
def recommend(selected_restaurant):
  topic = restaurant_df[restaurant_df['restaurant_name'] == selected_restaurant]['topic_id'].tolist()[0]
  # print(restaurant_df.groupby('topic_id').get_group(topic).sort_values(['ave_rating'], ascending=False)['restaurant_name'])
  restaurant_lst = restaurant_df.groupby('topic_id').get_group(topic).sort_values(['ave_rating'], ascending=False)['restaurant_name'].tolist()
  restaurant_lst = [r for r in restaurant_lst if r != selected_restaurant]
  if len(restaurant_lst) > 5: restaurant_lst = restaurant_lst[:5]
  return restaurant_lst

In [ ]:
selected_restaurant = '惡犬食堂'
recommended_restaurants = recommend(selected_restaurant)
print('Top recommended restaurants', '-----', recommended_restaurants, sep='\n')

Top recommended restaurants
-----
['大阪燒肉 燒魂Yakikon 大安店', '和牛涮 日式鍋物放題 台北羅斯福店', '和牛涮 日式鍋物放題 台北忠孝東店', '狗一下居食酒屋 忠孝店', '金洹苑 KIN KAN EN']
